In [30]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Задание на первую лабораторную: сделать классификацию тональности коротких текстов
Датасет: https://github.com/sismetanin/rureviews 
Оставим только negative и positive отзывы
- почистить текст, убрать знаки пунктуации (+токенизация)
- лемматизировать текст 
- сгенерировать коллокации
- использовать логистическую регрессию
- поверх регрессии использовать GridSearchCV или BayesSearchCV

In [31]:
df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", delimiter='\t')
df = df[df['sentiment'] != 'neautral']
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
df['review_processed'] = df['review_processed'].apply(lambda x: x.lower())
df['review_processed'] = df['review_processed'].apply(lambda x: word_tokenize(x))

morph = pymorphy2.MorphAnalyzer()

df['review_morphed'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])

100%|██████████| 60000/60000 [02:44<00:00, 364.33it/s]


In [32]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform([" ".join(x) for x in df['review_morphed']])
print(X.shape)

train_ds, test_ds, train_marks, test_marks = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

reg = LogisticRegression(max_iter=1000)
reg.fit(train_ds, train_marks)
predict = reg.predict_log_proba(test_ds)[:, 1]

print(roc_auc_score(test_marks, predict))

(60000, 51549)
0.9735222897024753


In [33]:
param_grid = {
    'C': [3.5, 3.2, 3.1, 3, 2.9],
    'max_iter': [150, 250, 500, 1000, 1500]
}

reg_gsCV = LogisticRegression(penalty='l2')
gsCV = GridSearchCV(reg_gsCV, param_grid, verbose=True, n_jobs=8)
gsCV.fit(train_ds, train_marks)
predict = gsCV.predict_log_proba(test_ds)[:, 1]

print("ROC:", roc_auc_score(test_marks, predict))
print(gsCV.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Unexpected exception formatting exception. Falling back to standard exception


joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\site-packages\joblib\externals\loky\process_executor.py", line 426, in _process_worker
    call_item = call_queue.get(block=True, timeout=timeout)
  File "c:\Users\emely\AppData\Local\Programs\Python\Python310\lib\multiprocessing\queues.py", line 122, in get
    return _ForkingPickler.loads(res)
ModuleNotFoundError: No module named 'sklearn.utils.parallel'
"""

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\emely\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\emely\AppData\Local\Temp\ipykernel_19968\3350847397.py", line 8, in <module>
    gsCV.fit(train_ds, train_marks)
  File "c:\Users\emely\AppData\Local\Programs\Python\Py

In [34]:
# df = pd.read_csv("women-clothing-accessories.3-class.balanced.csv", encoding="utf8", delimiter='\t')
# # фильтруем создаем новый DataFrame, который содержит только те строки из df, 
# # где значение в столбце 'sentiment' не равно 'neautral'.
# df = df[df['sentiment'] != 'neautral']
# # убрали знаки препинания
# df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
# # привели к нижнему регистру
# df['review_processed'] = df['review_processed'].apply(lambda x: x.lower())
# # разабъем слова
# df['review_processed'] = df['review_processed'].apply(lambda x: word_tokenize(x))

# # вернет информацию о его форме, грамматических характеристиках и т. д, лемантизируем
# morph = pymorphy2.MorphAnalyzer()
# df['review_morphed'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])

In [35]:
# # Создайте TF-IDF векторизатор (числовые признаки)
# tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1))
# X = tfidf_vectorizer.fit_transform([" ".join(x) for x in df['review_morphed']])

регрессия

In [36]:
# X_train, X_test, train_marks, y_test = train_test_split(X, df['sentiment'], test_size=0.2, random_state=42)

# reg = LogisticRegression(max_iter=1000)
# reg.fit(X_train, train_marks)
# predict = reg.predict_log_proba(X_test)[:, 1]

# print('Метрика - ', roc_auc_score(y_test, predict))

# # eli5.show_weights(estimator=reg, 
# #                   feature_names= list(tfidf_vectorizer.get_feature_names()),
# #                  top=(50, 50))

GridSearchCV

In [37]:
# train_ds, test_ds, train_marks, test_marks = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)
# param_grid = {
#     'C': [3.5, 3.2, 3.1, 3, 2.9],
#     'max_iter': [150, 250, 500, 1000, 1500]
# }

# reg_gsCV = LogisticRegression(penalty='l2')
# gsCV = GridSearchCV(reg_gsCV, param_grid, verbose=True, n_jobs=8)
# gsCV.fit(train_ds, train_marks)
# predict = gsCV.predict_log_proba(test_ds)[:, 1]

# print("ROC:", roc_auc_score(test_marks, predict))
# print(gsCV.best_params_)

BayesSearchCV